In [ ]:
import os
# CUDAVISIBLE DEVICES
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import transformers

In [ ]:
from snapkv.monkeypatch.monkeypatch import replace_llama, replace_mistral, replace_mixtral

In [ ]:
replace_mistral()

In [ ]:
from fastchat.model import load_model, get_conversation_template

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map="auto",
    use_flash_attention_2=True
    )

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

In [ ]:
with open('snapkv.txt', 'r') as f:
    content = f.read().strip()

In [ ]:
question = "\n What is the repository of SnapKV?"

In [ ]:
conv = get_conversation_template("mistralai/Mistral-7B-Instruct-v0.2")
conv.messages = []
conv.append_message(conv.roles[0],content + question)
# conv.append_message(conv.roles[0],"Who is Kobe Bryant?")
conv.append_message(conv.roles[1], None)

In [ ]:
prompt = conv.get_prompt()

In [ ]:
input_ids = tokenizer.encode(prompt, return_tensors='pt')

In [ ]:
input_ids_len = input_ids.size(1)
print(input_ids_len)

In [ ]:
outputs = model.generate(input_ids.cuda(), max_new_tokens=200, do_sample=False)

In [ ]:
print(tokenizer.decode(outputs[0][input_ids_len:], skip_special_tokens=True))